In [5]:
import wrds

# Establish connection
conn = wrds.Connection()

# Retrieve and display column information for comp.secm
table_info = conn.describe_table(library='comp', table='secm')
print(table_info)


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Approximately 8165980 rows in comp.secm.
         name  nullable            type  \
0       gvkey      True      VARCHAR(6)   
1         iid      True      VARCHAR(3)   
2    datadate      True            DATE   
3         tic      True      VARCHAR(8)   
4       cusip      True     VARCHAR(20)   
5        conm      True     VARCHAR(70)   
6       ajexm      True  NUMERIC(18, 8)   
7        ajpm      True  NUMERIC(18, 8)   
8      isalrt      True      VARCHAR(8)   
9     primiss      True      VARCHAR(1)   
10     cheqvm      True  NUMERIC(19, 4)   
11   curcddvm      True      VARCHAR(3)   
12     dvpspm      True  NUMERIC(19, 4)   
13     dvpsxm      True  NUMERIC(19, 4)   
14     dvrate      True  NUMERIC(19, 4)   
15      csfsm      True  NUMERIC(19, 4)   
16     cshtrm      True  NUMERIC(19

In [26]:
import wrds

# Establish a connection to WRDS
conn = wrds.Connection()

# Define the SIC codes for the energy sector
sic_codes = ('1311', '1381', '1382', '1389', '2911', '4922', '4923', '4924', '4931', '4932', '4939', '5171')

# Construct the SQL query
query = f"""
SELECT a.tic AS ticker,
       a.datadate AS date,
       a.prccm AS close,
       a.prchm AS high,
       a.prclm AS low,
       a.cshtrm AS volume,
       a.cshoq AS shares_outstanding,
       a.trt1m AS monthly_total_return ,
       a.dvpspm AS dividend_per_share,
       a.curcdm AS currency,
       b.sic AS sic_code,
       b.conm AS company_name
FROM comp.secm AS a
LEFT JOIN comp.company AS b
ON a.gvkey = b.gvkey
WHERE b.sic IN {sic_codes}
AND a.exchg IN (11, 14)  -- 11 = NYSE, 14 = NASDAQ
AND a.datadate BETWEEN '1994-01-01' AND '2025-01-01'
AND b.fic = 'USA'  -- Ensure only US stocks
"""

# Execute the query and fetch the data
energy_data = conn.raw_sql(query, date_cols=['date'])

# Close the WRDS connection
conn.close()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [27]:
energy_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74378 entries, 0 to 74377
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ticker                74378 non-null  string        
 1   date                  74378 non-null  datetime64[ns]
 2   close                 73890 non-null  Float64       
 3   high                  73890 non-null  Float64       
 4   low                   73890 non-null  Float64       
 5   volume                73714 non-null  Float64       
 6   shares_outstanding    22218 non-null  Float64       
 7   monthly_total_return  73916 non-null  Float64       
 8   dividend_per_share    15809 non-null  Float64       
 9   currency              73917 non-null  string        
 10  sic_code              74378 non-null  string        
 11  company_name          74378 non-null  string        
dtypes: Float64(7), datetime64[ns](1), string(4)
memory usage: 7.3 MB


In [28]:
df = energy_data.copy()
# Count null values for each column
null_counts = df.isnull().sum()

# Count the number of unique tickers
unique_tickers = df['ticker'].nunique()

# Display results
print("🔹 Null Values Per Column:\n")
print(null_counts)

print("\n🔹 Number of Unique Tickers:", unique_tickers)

🔹 Null Values Per Column:

ticker                      0
date                        0
close                     488
high                      488
low                       488
volume                    664
shares_outstanding      52160
monthly_total_return      462
dividend_per_share      58569
currency                  461
sic_code                    0
company_name                0
dtype: int64

🔹 Number of Unique Tickers: 563


In [29]:
# Drop rows where 'close', 'high', or 'low' have null values
df_cleaned = df.dropna(subset=['close', 'high', 'low','monthly_total_return','volume'])

# Count remaining null values after dropping
null_counts_after = df_cleaned.isnull().sum()

# Count unique tickers after cleaning
unique_tickers_after = df_cleaned['ticker'].nunique()

# Save the cleaned dataset to a new CSV file
cleaned_file_path = "C:\\Users\\Jrans\\Desktop\\Model Comparison Project\\energy_stocks_cleaned.csv"
df_cleaned.to_csv(cleaned_file_path, index=False)

# Display results
print("🔹 Null Values After Cleaning:\n")
print(null_counts_after)

print("\n🔹 Number of Unique Tickers After Cleaning:", unique_tickers_after)
print(f"\n✅ Cleaned dataset saved to: {cleaned_file_path}")

🔹 Null Values After Cleaning:

ticker                      0
date                        0
close                       0
high                        0
low                         0
volume                      0
shares_outstanding      51302
monthly_total_return        0
dividend_per_share      57663
currency                    0
sic_code                    0
company_name                0
dtype: int64

🔹 Number of Unique Tickers After Cleaning: 557

✅ Cleaned dataset saved to: C:\Users\Jrans\Desktop\Model Comparison Project\energy_stocks_cleaned.csv


In [30]:
df_cleaned.describe()

,date,close,high,low,volume,shares_outstanding,monthly_total_return,dividend_per_share
count,73405,73405.0,73405.0,73405.0,73405.0,22103.0,73405.0,15742.0
mean,2008-04-09 14:43:15.844969728,31.035564,33.011529,28.780955,26517286.266317,178.685121,14.413823,0.361897
min,1994-01-31 00:00:00,0.001,0.001,0.001,10.0,0.436,-99.99,0.0
25%,2000-01-31 00:00:00,13.5,14.88,12.05,1135400.0,23.81,-4.5597,0.1625
50%,2007-10-31 00:00:00,25.4,26.98,23.625,5803900.0,59.794,0.7871,0.3125
75%,2015-11-30 00:00:00,41.3,43.72,38.44,24354908.0,174.0325,6.3236,0.4822
max,2024-12-31 00:00:00,277.94,288.46,263.69,1643903195.0,6840.0,949900.0,37.3868
std,NaN,25.56022,26.799268,23.996743,61133062.742774,429.022923,3506.056182,0.471525
